In [ ]:
!pip install tqdm

In [ ]:
import gymnasium as gym
import numpy as np
import math
import random
from gymnasium import Env, spaces
from gymnasium.spaces import  Discrete
import numpy as np
import random
from scipy.stats import bernoulli
import yaml
from matplotlib import colors
from tqdm import trange
from matplotlib import pyplot as plt

In [ ]:
class RandomMaze(Env):
    def __init__(self):
        self.start_state = int(8)
        self.goal = int(3)
        self.hole = int(7)
        self.state = int(8)
        self.action_space = spaces.Discrete(4)
        self.observation_space = spaces.Discrete(12)
        self.P = {
            0: {
                0: [(0.8, 0, -0.04, False), (0.1, 0, -0.04, False), (0.1, 4, -0.04, False)],
                1: [(0.8, 0, -0.04, False), (0.1, 0, -0.04, False), (0.1, 1, -0.04, False)],
                2: [(0.8, 1, -0.04, False), (0.1, 0, -0.04, False), (0.1, 4, -0.04, False)],
                3: [(0.8, 4, -0.04, False), (0.1, 0, -0.04, False), (0.1, 1, -0.04, False)],
            },
            1: {
                0: [(0.8, 0, -0.04, False), (0.1, 1, -0.04, False), (0.1, 1, -0.04, False)],
                1: [(0.8, 1, -0.04, False), (0.1, 0, -0.04, False), (0.1, 2, -0.04, False)],
                2: [(0.8, 2, -0.04, False), (0.1, 1, -0.04, False), (0.1, 1, -0.04, False)],
                3: [(0.8, 1, -0.04, False), (0.1, 0, -0.04, False), (0.1, 2, -0.04, False)],
            },
            2: {
                0: [(0.8, 1, -0.04, False), (0.1, 2, -0.04, False), (0.1, 6, -0.04, False)],
                1: [(0.8, 2, -0.04, False), (0.1, 1, -0.04, False), (0.1, 3, +1, True)],
                2: [(0.8, 3, +1, True), (0.1, 2, -0.04, False), (0.1, 6, -0.04, False)],
                3: [(0.8, 6, -0.04, False), (0.1, 1, -0.04, False), (0.1, 3, +1, True)],
            },
            3: {
                0: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
                1: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
                2: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
                3: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
            },
            4: {
                0: [(0.8, 4, -0.04, False), (0.1, 0, -0.04, False), (0.1, 8, -0.04, False)],
                1: [(0.8, 0, -0.04, False), (0.1, 4, -0.04, False), (0.1, 4, -0.04, False)],
                2: [(0.8, 4, -0.04, False), (0.1, 0, -0.04, False), (0.1, 8, -0.04, False)],
                3: [(0.8, 8, -0.04, False), (0.1, 4, -0.04, False), (0.1, 4, -0.04, False)],
            },
            6: {
                0: [(0.8, 6, -0.04, False), (0.1, 2, -0.04, False), (0.1, 10, -0.04, False)],
                1: [(0.8, 2, -0.04, False), (0.1, 6, -0.04, False), (0.1, 7, -1, True)],
                2: [(0.8, 7, -1, True), (0.1, 2, -0.04, False), (0.1, 10, -0.04, False)],
                3: [(0.8, 10, -0.04, False), (0.1, 6, -0.04, False), (0.1, 7, -1, True)],
            },
            7: {
                0: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
                1: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
                2: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
                3: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
            },
            8: {
                0: [(0.8, 8, -0.04, False), (0.1, 8, -0.04, False), (0.1, 4, -0.04, False)],
                1: [(0.8, 4, -0.04, False), (0.1, 8, -0.04, False), (0.1, 9, -0.04, False)],
                2: [(0.8, 9, -0.04, False), (0.1, 8, -0.04, False), (0.1, 4, -0.04, False)],
                3: [(0.8, 8, -0.04, False), (0.1, 8, -0.04, False), (0.1, 9, -0.04, False)],
            },
            9: {
                0: [(0.8, 8, -0.04, False), (0.1, 9, -0.04, False), (0.1, 9, -0.04, False)],
                1: [(0.8, 9, -0.04, False), (0.1, 8, -0.04, False), (0.1, 10, -0.04, False)],
                2: [(0.8, 10, -0.04, False), (0.1, 9, -0.04, False), (0.1, 9, -0.04, False)],
                3: [(0.8, 9, -0.04, False), (0.1, 8, -0.04, False), (0.1, 10, -0.04, False)]
            },
            10: {
                0: [(0.8, 9, -0.04, False), (0.1, 6, -0.04, False), (0.1, 10, -0.04, False)],
                1: [(0.8, 6, -0.04, False), (0.1, 9, -0.04, False), (0.1, 11, -0.04, False)],
                2: [(0.8, 11, -0.04, False), (0.1, 6, -0.04, False), (0.1, 10, -0.04, False)],
                3: [(0.8, 10, -0.04, False), (0.1, 9, -0.04, False), (0.1, 11, -0.04, False)]
            },
            11: {
                0: [(0.8, 10, -0.04, False), (0.1, 7, -1, True), (0.1, 11, -0.04, False)],
                1: [(0.8, 7, -1, True), (0.1, 10, -0.04, False), (0.1, 11, -0.04, False)],
                2: [(0.8, 11, -0.04, False), (0.1, 7, -1, True), (0.1, 11, -0.04, False)],
                3: [(0.8, 11, -0.04, False), (0.1, 11, -0.04, False), (0.1, 10, -0.04, False)]
            }
        }

    def seed(self, seed):
        np.random.seed(seed)
        return seed

    def step(self, action):
        d = np.random.choice([0, 1, 2], p=[0.8, 0.1, 0.1])
        tr = self.P[int(self.state)][int(action)][int(d)]
        prob = tr[0]
        self.state = int(tr[1])
        reward = tr[2]
        done = tr[3]
        return self.state, reward, done,False, {}


    def  reset(self, seed=None, options=None):
        # reset state
        super().reset(seed=seed)
        self.state=self.start_state 
        done=False
        info = {}
        return self.state,info

In [ ]:
from gymnasium import Env, register

register(id='RME-v0', entry_point=RandomMaze)

In [ ]:
env=gym.make('RME-v0')

In [ ]:
gamma = 0.99
alpha = 0.05
alpha_final = 0.01
epsilon = 0.37
decay_rate = 0.05
maxSteps = 50
noEpisodes = 500
noPlanning=5
maxTrajectory = 10

nStates = 12
nActions = 4
smooth_iters = 10
interesting_states = [0, 1, 2, 4, 6, 8, 9, 10, 11]
action_dict = {'0':"LEFT", '1':"UP", '2':"RIGHT", '3':"DOWN"}
action_list = ["LEFT", "UP", "RIGHT", "DOWN"]

In [ ]:

def plot_state_values(V, episodes=None, prefix=''):
    states_to_plot = [0, 1, 2, 4, 6, 8, 9, 10, 11]
    true_v_values = np.array([0.824, 0.893, 0.955, 0.0, 0.764, 0.0, 0.688, 0.0, 0.698, 0.639, 0.606, 0.382])
    if episodes is None:
        episodes = np.arange(V.shape[0])

    plt.figure(figsize=(15, 10))
    colors = plt.cm.tab10(np.linspace(0, 1, len(states_to_plot)))

    for idx, state in enumerate(states_to_plot):
        plt.plot(episodes, V[:, state], label=f'State {state}', color=colors[idx])
        if true_v_values is not None:
            plt.plot(episodes, [true_v_values[state]]*len(episodes), linestyle='--', label=f'True Value State {state}', color=colors[idx], alpha=0.5)

    plt.title("State-Value (V) Function vs. Episodes", fontsize=16)
    plt.xlabel("Episodes", fontsize=14)
    plt.ylabel("State Values", fontsize=14)
    plt.legend(fontsize=12)
    plt.grid(True)
    plt.savefig(f"{prefix}_state_values.pdf", bbox_inches='tight')
    plt.show()
    


def plot_action_values(Q, episodes=None, prefix=''):
    if episodes is None:
        episodes = np.arange(Q.shape[0])

    states_to_plot = [0, 1, 2, 4, 6, 8, 9, 10, 11]
    actions = ["LEFT", "UP", "RIGHT", "DOWN"]
    num_actions = len(actions)
    # Adjust the subplot layout based on the number of actions
    cols = 2  # You can adjust this to change how many columns of subplots you want
    rows = (num_actions + cols - 1) // cols  # Calculate rows needed based on the number of actions
    fig, axs = plt.subplots(rows, cols, figsize=(15, 5 * rows))
    if num_actions == 1:
        axs = [axs]  # Make axs iterable if there's only one plot
    axs = axs.flatten()
    
    for action_idx, action in enumerate(actions):
        ax = axs[action_idx]
        for state in states_to_plot:
            ax.plot(episodes, Q[:, state, action_idx], label=f'State {state}, Action {action}')
        ax.set_title(f"Action {action}")
        ax.set_xlabel("Episodes")
        ax.set_ylabel("Q Values")
        ax.legend()
    
    # Hide any unused subplots
    for ax in axs[num_actions:]:
        ax.set_visible(False)
    
    plt.tight_layout()
    plt.savefig(f"{prefix}_action_values.pdf", bbox_inches='tight')
    plt.show()
    

def plot_state_action_values(Q, episodes=None, prefix=''):
    if episodes is None:
        episodes = np.arange(Q.shape[0])

    interesting_states = [0, 1, 2, 4, 6, 8, 9, 10, 11]
    actions = ["LEFT", "UP", "RIGHT", "DOWN"]

    for state in interesting_states:
        fig, ax = plt.subplots(figsize=(8, 6))
        for action_idx, action in enumerate(actions):
            ax.plot(episodes, Q[:, state, action_idx], label=f'Action {action}')
        ax.set_title(f"State {state}: Q-Values across Actions")
        ax.set_xlabel("Episodes")
        ax.set_ylabel("Q-Values")
        ax.legend()
        plt.savefig(f"{prefix}_state{state}_action_values.pdf", bbox_inches='tight')
        plt.show()
        



def visualize_state_and_action_values(state_values, action_values):
    # Define optimal state values for comparison
    benchmark_values = np.array([0.824, 0.893, 0.955, 0.0, 0.764, 0.0, 0.688, 0.0, 0.698, 0.639, 0.606, 0.382])
    
    # Define a new palette of colors using HEX codes for uniqueness
    color_palette = ['#483D8B', '#32CD32', '#FF4500', '#008080', '#9932CC', '#2E8B57', '#FFA500', '#8B4513', '#6B8E23', '#4682B4', '#696969', '#FF69B4']
    
    episode_range = np.arange(len(state_values))
    
    plt.rcParams['figure.figsize'] = (15, 10)
    plt.cla()
    for state_index in range(len(benchmark_values)):
        if state_index == 5:  # Skipping a specific state if needed
            continue
        plt.plot(episode_range, state_values[:, state_index], label=f'State {state_index}', color=color_palette[state_index])
        plt.plot(episode_range, [benchmark_values[state_index]]*len(state_values), color=color_palette[state_index], linestyle='--')
    plt.legend(loc=(1.01,0))
    plt.title("State-Value (V) Function vs. Episodes Across States")
    plt.xlabel('Episode Count')
    plt.ylabel('State Values')
    plt.show()
    
    for selected_state in interesting_states:
        episode_range = np.arange(len(state_values))
        legend = []
        
        plt.cla()
        for action_index in range(action_values.shape[2]):  # Assuming action_values shape is [episodes, states, actions]
            plt.plot(episode_range, action_values[:, selected_state, action_index], label=f'Action {action_index}')
            legend.append(f"Action {action_index}")
        plt.legend(legend, loc='best')
        plt.title(f"Action-Value (Q) Function vs. Episodes for State {selected_state}")
        plt.xlabel('Episode Count')
        plt.ylabel('Action Values')
        plt.show()




In [ ]:

def generateTrajectory(env, Q, epsilon, max_steps=1000):
    steps = 0
    done = False
    traj = []

    # Assuming the reset method returns the initial state which is ignored here
    # If needed, capture the initial state as: state = env.reset()
    env.reset()

    while not done and steps < max_steps:
        old_state = env.state
        # Epsilon-greedy action selection
        if random.random() > epsilon:
            action = np.argmax(Q[old_state])
        else:
            action = np.random.randint(0, env.action_space.n)  # Assuming env.action_space.n is valid

        new_state, reward, done, truncated, info = env.step(action)
        traj.append((old_state, action, reward, new_state))
        
        steps += 1

    return traj


In [ ]:
def decayAlpha(initialValue, finalValue, maxSteps, decayType):
    step_size=np.zeros(maxSteps)
    if decayType=="linear":
        decayRate=(initialValue-finalValue)/(maxSteps -1)
        for i in range(maxSteps):
            step_size[i]=initialValue-i*decayRate

    else:
        decayRate=(np.log(initialValue/finalValue))/maxSteps
        step_size[0]=initialValue
        for i in range(maxSteps-1):
            step_size[i+1]=step_size[i]*np.exp(-decayRate)

    return step_size

In [ ]:
def decay_epsilon(epsilon, decay_rate, decay_type):
    min_value = 0.0
    if decay_type == 'linear':
        epsilon = max(min_value, epsilon-decay_rate*epsilon)
    else:
        epsilon = max(min_value, epsilon*np.exp(-1*decay_rate))
    return epsilon

In [ ]:
def monte_carlo_control(env, gamma, alpha, epsilon, maxSteps, noEpisodes, firstVisit = True):
    Q = np.zeros((nStates, nActions))
    Qe = np.zeros((noEpisodes, nStates, nActions))
    alphas = decayAlpha(alpha, alpha_final, noEpisodes, 'linear')
    for ep in range(noEpisodes):
        alpha = alphas[ep]
        epsilon = decay_epsilon(epsilon, decay_rate, 'linear')
        traj = generateTrajectory(env, Q, epsilon, maxSteps)
        visited = np.zeros((nStates, nActions))
        for i, (old_state, action, reward, new_state) in enumerate(traj):
            if visited[old_state, action] != 0 and firstVisit:
                continue

            visited[old_state, action] = 1

            g_t = 0
            for j in range(i, len(traj)):
                reward = traj[j][2]
                g_t += (gamma**(j-i)) * reward 
            G = g_t
            Q[old_state, action] += alpha * (G - Q[old_state, action])
        Qe[ep] = Q
    state_value = np.max(Qe, axis=-1)
    q_value = Qe
    optimal_policy = np.argmax(Q, axis=-1)
        
    return state_value, q_value, optimal_policy

In [ ]:
state_value_all = np.zeros((smooth_iters, noEpisodes, nStates))
q_value_all = np.zeros((smooth_iters, noEpisodes, nStates, nActions))
for it in trange(smooth_iters, desc='Smoothing'):
    env.seed(it)
    state_value, q_value, optimal_policy = monte_carlo_control(env, gamma, alpha, epsilon, maxSteps, noEpisodes, firstVisit = True)
    state_value_all[it] = state_value
    q_value_all[it] = q_value
state_value = np.average(state_value_all, axis=0)
q_value = np.average(q_value_all, axis=0)
'''
plot_state_values(state_value,  episodes=range(noEpisodes))
plot_action_values(q_value,  episodes=range(noEpisodes))
plot_state_action_values(q_value,  episodes=range(noEpisodes))
'''
plot_state_values(state_value,  episodes=range(noEpisodes), prefix='mcc-')
plot_action_values(q_value,  episodes=range(noEpisodes), prefix='mcc-')
plot_state_action_values(q_value,  episodes=range(noEpisodes), prefix='mcc-')


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

def visualize_policy(grid_policy):
    reshaped_policy = np.array(grid_policy).reshape((3, 4))
    
    # Define a modern color map
    cmap = mcolors.ListedColormap(['#f0f0f0', '#f0f0f0'])  # Light grey for the grid
    fig, ax = plt.subplots(figsize=(8, 6))  # Adjust for a more aesthetic aspect ratio
    ax.imshow(reshaped_policy, cmap=cmap)
    
    # Set up grid aesthetics
    ax.set_xticks(np.arange(4))
    ax.set_yticks(np.arange(3))
    ax.set_xticks(np.arange(-.5, 4, 1), minor=True)
    ax.set_yticks(np.arange(-.5, 3, 1), minor=True)
    ax.grid(which='minor', color='#333333', linestyle='-', linewidth=2)  # Darker grid lines for contrast
    ax.tick_params(which='both', bottom=False, left=False, labelleft=False, labelbottom=False)  # Hide ticks
    
    # Annotate cells with directions or special markers
    directions = {0: '←', 1: '↑', 2: '→', 3: '↓'}
    special_states = {5: 'W', 3: 'G', 7: 'H'}  # Define special states separately for clarity
    for y in range(3):
        for x in range(4):
            state = y*4 + x
            cell_value = reshaped_policy[y, x]
            text = special_states.get(state, directions.get(cell_value, ''))
            ax.text(x, y, text, ha='center', va='center', color='#007acc', fontsize=15, weight='bold')  # Use a bold, blue font for visibility
    
    plt.show()


In [ ]:
smooth_optimal_policy = np.argmax(q_value[-1], axis=-1)
visualize_policy(smooth_optimal_policy)

In [ ]:
def actionSelect(state, Q, epsilon):
    u = np.random.uniform()
    if u > epsilon:
        action = np.argmax(Q[env.state])
    else:
        action = np.random.randint(0, nActions)
    return action

In [ ]:
def sarsa(env, gamma, alpha, epsilon, noEpisodes):
    Q = np.zeros((nStates, nActions))
    Qe = np.zeros((noEpisodes, nStates, nActions))
    alphas = decayAlpha(alpha, alpha_final, noEpisodes, 'linear')
    for ep in range(noEpisodes):
        alpha = alphas[ep]
        epsilon = decay_epsilon(epsilon, decay_rate, 'linear')
        
        old_state,done = env.reset()
        old_action = actionSelect(old_state, Q, epsilon)        
        
        while not done:            
            new_state, reward, done,truncated,info = env.step(old_action)
            new_action = actionSelect(new_state, Q, epsilon)
            td_target = reward
            
            if not done:
                td_target = td_target + gamma * Q[new_state, new_action]
            
            td_error = td_target - Q[old_state, old_action]

            Q[old_state, old_action] = Q[old_state, old_action] + alpha * td_error
            old_state = new_state
            old_action = new_action                
        Qe[ep, :] = Q        
    
    V = np.max(Qe, axis = 2)
    optimal_policy = np.amax(Qe, axis = 2)
        
    state_value = V
    q_value = Qe
    return state_value, q_value, optimal_policy

In [ ]:
state_value_all = np.zeros((smooth_iters, noEpisodes, nStates))
q_value_all = np.zeros((smooth_iters, noEpisodes, nStates, nActions))
for it in trange(smooth_iters, desc='Smoothing'):
    env.seed(it)
    state_value, q_value, optimal_policy = sarsa(env, gamma, alpha, epsilon, noEpisodes)
    state_value_all[it] = state_value
    q_value_all[it] = q_value
state_value = np.average(state_value_all, axis=0)
q_value = np.average(q_value_all, axis=0)
plot_state_values(state_value,  episodes=range(noEpisodes), prefix= 'sarsa-')
plot_action_values(q_value,  episodes=range(noEpisodes), prefix='sarsa-')
plot_state_action_values(q_value,  episodes=range(noEpisodes), prefix='sarsa-')


In [ ]:
smooth_optimal_policy = np.argmax(q_value[-1], axis=-1)
visualize_policy(smooth_optimal_policy)

In [ ]:
def q_learning(env, gamma, alpha, epsilon, noEpisodes):
    Q = np.zeros((nStates, nActions))
    Qe = np.zeros((noEpisodes, nStates, nActions))
    alphas = decayAlpha(alpha, alpha_final, noEpisodes, 'linear')
    for ep in range(noEpisodes):
        alpha = alphas[ep]
        epsilon = decay_epsilon(epsilon, decay_rate, 'linear')
        
        old_state,done = env.reset()
        
        while not done:
            old_action = actionSelect(old_state, Q, epsilon)
            new_state, reward, done, truncated, info= env.step(old_action)
            td_target = reward
            
            if not done:
                td_target = td_target + gamma * Q[new_state].max()
            
            td_error = td_target - Q[old_state, old_action]

            Q[old_state, old_action] = Q[old_state, old_action] + alpha * td_error
            old_state = new_state                
        Qe[ep, :] = Q
        
    
    V = np.max(Qe, axis = 2)
    optimal_policy = np.amax(Qe, axis = 2)
        
    state_value = V
    q_value = Qe
    return state_value, q_value, optimal_policy

In [ ]:
state_value_all = np.zeros((smooth_iters, noEpisodes, nStates))
q_value_all = np.zeros((smooth_iters, noEpisodes, nStates, nActions))
for it in trange(smooth_iters, desc='Smoothing'):
    env.seed(it)
    state_value, q_value, optimal_policy = q_learning(env, gamma, alpha, epsilon, noEpisodes)
    state_value_all[it] = state_value
    q_value_all[it] = q_value
state_value = np.average(state_value_all, axis=0)
q_value = np.average(q_value_all, axis=0)
plot_state_values(state_value,  episodes=range(noEpisodes) , prefix='ql-')
plot_action_values(q_value,  episodes=range(noEpisodes), prefix='ql-')
plot_state_action_values(q_value,  episodes=range(noEpisodes), prefix='ql-')

In [ ]:
smooth_optimal_policy = np.argmax(q_value[-1], axis=-1)
visualize_policy(smooth_optimal_policy)

In [ ]:
def double_q_learning(env, gamma, alpha, epsilon, noEpisodes):
    Q = np.zeros((nStates, nActions))
    Qe = np.zeros((noEpisodes, nStates, nActions))
    
    Q_1 = np.zeros((nStates, nActions))
    Qe_1 = np.zeros((noEpisodes, nStates, nActions))
    
    Q_2 = np.zeros((nStates, nActions))
    Qe_2 = np.zeros((noEpisodes, nStates, nActions))
    
    alphas = decayAlpha(alpha, alpha_final, noEpisodes, 'linear')
    
    for ep in range(noEpisodes):
        alpha = alphas[ep]
        epsilon = decay_epsilon(epsilon, decay_rate, 'linear')
        
        old_state,done = env.reset()

        while not done:
            old_action = actionSelect(old_state, Q_1, epsilon)
            new_state, reward, done,truncated,info = env.step(old_action)
            td_target = reward
            
            if np.random.randint(2):
                action_q1 = np.argmax(Q_1[new_state])
                td_target    = reward
                
                if not done:
                    td_target    = td_target + gamma * Q_2[new_state, action_q1]
            
                td_error     = td_target - Q_1[old_state, old_action]

                Q_1[old_state, old_action] = Q_1[old_state, old_action] + alpha * td_error
                
                
            else:
                action_q2 = np.argmax(Q_2[new_state])
                td_target    = reward
                
                if not done:
                    td_target    = td_target + gamma * Q_1[new_state, action_q2]
            
                td_error     = td_target - Q_2[old_state, old_action]

                Q_2[old_state, old_action] = Q_2[old_state, old_action] + alpha * td_error
            old_state = new_state                
        Qe_1[ep, :] = Q_1
        Qe_2[ep, :] = Q_2
        Q           = (Q_1 + Q_2)/2
        Qe          = (Qe_1 + Qe_2)/2

    V = np.max(Qe, axis = 2)
    optimal_policy = np.amax(Qe, axis = 2)
        
    state_value = V
    q_value = Qe
    return state_value, q_value, optimal_policy

In [ ]:
state_value_all = np.zeros((smooth_iters, noEpisodes, nStates))
q_value_all = np.zeros((smooth_iters, noEpisodes, nStates, nActions))
for it in trange(smooth_iters, desc='Smoothing'):
    env.seed(it)
    state_value, q_value, optimal_policy = double_q_learning(env, gamma, alpha, epsilon, noEpisodes)
    state_value_all[it] = state_value
    q_value_all[it] = q_value
state_value = np.average(state_value_all, axis=0)
q_value = np.average(q_value_all, axis=0)
plot_state_values(state_value,  episodes=range(noEpisodes), prefix='dql-')
plot_action_values(q_value,  episodes=range(noEpisodes), prefix='dql-')
plot_state_action_values(q_value,  episodes=range(noEpisodes), prefix='dql-')

In [ ]:
smooth_optimal_policy = np.argmax(q_value[-1], axis=-1)
visualize_policy(smooth_optimal_policy)

In [ ]:
def get_policy_success_rate(env, current_policy, goal_state, maxEpisodes = 100, maxSteps = 200):
    success = 0
    for _ in range(maxEpisodes):
        env.reset()
        for _ in range(maxSteps):
            env.step(current_policy[env.state])
            if env.state == 3 or env.state == 7:
                break
        if env.state == goal_state:
            success += 1
    success /= maxEpisodes
    policy_success_rate = success * 100
    
    return policy_success_rate   

In [ ]:
def plot_success_rate():
    q_value_all_mc = np.zeros((smooth_iters, noEpisodes, nStates, nActions))
    q_value_all_sarsa = np.zeros((smooth_iters, noEpisodes, nStates, nActions))
    q_value_all_q = np.zeros((smooth_iters, noEpisodes, nStates, nActions))
    q_value_all_dq = np.zeros((smooth_iters, noEpisodes, nStates, nActions))
    
    for it in trange(smooth_iters, desc='Smoothing'):
        env.seed(it)
        _, q_value_mc, _ = monte_carlo_control(env, gamma, alpha, epsilon, maxSteps, noEpisodes, firstVisit = True)
        _, q_value_sarsa, _ = sarsa(env, gamma, alpha, epsilon, noEpisodes)
        _, q_value_q, _ = q_learning(env, gamma, alpha, epsilon, noEpisodes)
        _, q_value_dq, _ = double_q_learning(env, gamma, alpha, epsilon, noEpisodes)
        
        q_value_all_mc[it] = q_value_mc
        q_value_all_sarsa[it] = q_value_sarsa
        q_value_all_q[it] = q_value_q
        q_value_all_dq[it] = q_value_dq
    
    q_value_mc = np.average(q_value_all_mc, axis=0)
    q_value_sarsa = np.average(q_value_all_sarsa, axis=0)
    q_value_q = np.average(q_value_all_q, axis=0)
    q_value_dq = np.average(q_value_all_dq, axis=0)

    optimal_policies_mc = np.argmax(q_value_mc, axis=-1)
    optimal_policies_sarsa = np.argmax(q_value_sarsa, axis=-1)
    optimal_policies_q = np.argmax(q_value_q, axis=-1)
    optimal_policies_dq = np.argmax(q_value_dq, axis=-1)
    
    success_rates_mc = np.zeros(noEpisodes)
    success_rates_sarsa = np.zeros(noEpisodes)
    success_rates_q = np.zeros(noEpisodes)
    success_rates_dq = np.zeros(noEpisodes)
    
    for i in trange(noEpisodes, desc='Episodes'):
        success_rates_mc[i] = get_policy_success_rate(env, optimal_policies_mc[i], 3)
        success_rates_sarsa[i] = get_policy_success_rate(env, optimal_policies_sarsa[i], 3)
        success_rates_q[i] = get_policy_success_rate(env, optimal_policies_q[i], 3)
        success_rates_dq[i] = get_policy_success_rate(env, optimal_policies_dq[i], 3)

    plt.rcParams['figure.figsize'] = (12, 8)
    plt.cla()
    color_scheme = {
    'FVMCC': '#FF5733',  # A bright orange
    'SARSA': '#33FF57',  # A vivid green
    'QL':    '#3357FF',  # A deep blue
    'DQL':   '#F033FF',  # A vibrant purple
     }
    plt.plot(np.arange(noEpisodes), success_rates_mc, label='FVMCC',  color=color_scheme['FVMCC'])
    plt.plot(np.arange(noEpisodes), success_rates_sarsa, label='SARSA',  color=color_scheme['SARSA'])
    plt.plot(np.arange(noEpisodes), success_rates_q, label='QL', color=color_scheme['QL'])
    plt.plot(np.arange(noEpisodes), success_rates_dq, label='DQL', color=color_scheme['DQL'])
    plt.legend()
    plt.xlabel('Episodes')
    plt.ylabel('Success Rate')
    plt.title("Success Rate vs. Episodes")
    plt.savefig(f"succratevseps.pdf", bbox_inches='tight')
    plt.show()

def plot_estimated_returns_and_mae():
    optimal_values = np.asarray([0.82442985, 0.89286374, 0.95464233, 0., 0.76427487, 0.,0.68820946, 0., 0.69763948, 0.63906542, 0.60613373, 0.38186228])
    
    state_value_all_mc = np.zeros((smooth_iters, noEpisodes, nStates))
    state_value_all_sarsa = np.zeros((smooth_iters, noEpisodes, nStates))
    state_value_all_q = np.zeros((smooth_iters, noEpisodes, nStates))
    state_value_all_dq = np.zeros((smooth_iters, noEpisodes, nStates))
    
    for it in trange(smooth_iters, desc='Smoothing'):
        env.seed(it)
        state_value_mc, _, _ = monte_carlo_control(env, gamma, alpha, epsilon, maxSteps, noEpisodes, firstVisit = True)
        state_value_sarsa, _, _ = sarsa(env, gamma, alpha, epsilon, noEpisodes)
        state_value_q, _, _ = q_learning(env, gamma, alpha, epsilon, noEpisodes)
        state_value_dq, _, _ = double_q_learning(env, gamma, alpha, epsilon, noEpisodes)
        
        state_value_all_mc[it] = state_value_mc
        state_value_all_sarsa[it] = state_value_sarsa
        state_value_all_q[it] = state_value_q
        state_value_all_dq[it] = state_value_dq
    
    state_value_mc = np.average(state_value_all_mc, axis=0)
    state_value_sarsa = np.average(state_value_all_sarsa, axis=0)
    state_value_q = np.average(state_value_all_q, axis=0)
    state_value_dq = np.average(state_value_all_dq, axis=0)

    plt.rcParams['figure.figsize'] = (12, 8)
    x = np.arange(noEpisodes)
    plt.cla()

    plt.plot(x, state_value_mc[:, 8], label='FVMCC', color=color_scheme['FVMCC'])
    plt.plot(x, state_value_sarsa[:, 8], label='SARSA', color=color_scheme['SARSA'])
    plt.plot(x, state_value_q[:, 8], label='QL', color=color_scheme['QL'])
    plt.plot(x, state_value_dq[:, 8], label='DQL', color=color_scheme['DQL'])
    plt.legend()
    plt.xlabel('Episodes')
    plt.ylabel('Value')
    plt.title("Estimated Expected Return (from the Start State) vs. Episodes")
    plt.savefig(f"expectedreturnvseps.pdf", bbox_inches='tight')
    plt.show()

    mae_mc = np.average(np.abs(state_value_mc - optimal_values.reshape(1, -1)), axis=-1)
    mae_sarsa = np.average(np.abs(state_value_sarsa - optimal_values.reshape(1, -1)), axis=-1)
    mae_q = np.average(np.abs(state_value_q - optimal_values.reshape(1, -1)), axis=-1)
    mae_dq = np.average(np.abs(state_value_dq - optimal_values.reshape(1, -1)), axis=-1)

    plt.cla()
    plt.plot(x, mae_mc, label='FVMCC', color=color_scheme['FVMCC'])
    plt.plot(x, mae_sarsa, label='SARSA', color=color_scheme['SARSA'])
    plt.plot(x, mae_q, label='QL', color=color_scheme['QL'])
    plt.plot(x, mae_dq, label='DQL', color=color_scheme['DQL'])
    plt.legend()
    plt.xlabel('Episodes')
    plt.ylabel('MAE')
    plt.title("State-Value Function Estimation Error vs. Episodes")
    plt.savefig(f"statevaluefvseps.pdf", bbox_inches='tight')
    plt.show()

plot_success_rate()
plot_estimated_returns_and_mae()